In [1]:
import requests
import time
import json
from IPython.display import JSON

Obtain `issn_l` from OpenAlex for Open Access journals

In [5]:
f=open('/scratch/restrepo/oa.json','r')
va=json.load(f)
f.close()

In [6]:
issns=[d.get('issn_l') for d in va]

In [7]:
len(issns)

14383

Load DOAJ journal full json database

In [2]:
f=open('/scratch/restrepo/doaj.json','r')
doaj=json.load(f)
f.close()

In [ ]:
ls -lh /scratch/restrepo/doaj.json

-rw-r--r-- 1 restrepo restrepo 29M Oct 30 09:30 /scratch/restrepo/doaj.json


Build backend for DOAJ

In [88]:
DOAJ={}
i=0
for issn in issns:
    loa=[d['bibjson'] for d in doaj if d['bibjson'].get('eissn')==issn or d['bibjson'].get('pissn')==issn]
    print(str(i).zfill(5),end='\r')
    i+=1
    if not loa:
        continue
    
    d=loa[0]
    
    try:
        doi = 'DOI' in d.get('pid_scheme').get('scheme')
    except:
        doi = False
    try:
        license = d.get('license')[0].get('type')
    except:
        license = None


    DOAJ[issn] = {'issn_l':issn,
        'publication_time_weeks': d.get('publication_time_weeks'),
     'oa_start': d.get( 'oa_start'),
     'copyright.author_retains': d.get('copyright').get('author_retains'),
     'editorial.review_process': d.get('editorial').get('review_process'),
     'other_charges.has_other_charges': d.get('other_charges').get('has_other_charges'),
     'pid_scheme.DOI': doi,
     'plagiarism.detection': d.get('plagiarism').get('detection'),
     'waiver.has_waiver': d.get('waiver').get('has_waiver'),
     'keywords': d.get('keywords'),
     'language': d.get('language'),
     'license.type': license,
     'subject.term': [ x.get('term')  for x in d.get('subject')][::-1]
    }        

In [75]:
len(DOAJ.keys())

13128

In [90]:
f=open('DOAJ.json','w')
json.dump(DOAJ,f)
f.close()

In [91]:
ls -lh DOAJ.json

-rw-r--r-- 1 restrepo restrepo 6.4M Oct 30  2023 DOAJ.json


In [93]:
DOAJ.get('2688-8130')